In [18]:
import pandas as pd
train_df = pd.read_csv('C:/coding/mortality_project/data/preprocessed_train_delta.csv', encoding='cp949')
valid_df = pd.read_csv('C:/coding/mortality_project/data/preprocessed_valid_delta.csv', encoding='cp949')
test_df = pd.read_csv('C:/coding/mortality_project/data/preprocessed_test_delta.csv', encoding='cp949')

In [2]:
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
valid_df.drop(['Unnamed: 0'], axis=1, inplace=True)
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
def to_secs(rec):
    dh, m, s = rec.split(':')
    if 'days' in dh:
        d, h = dh.split(' days, ')
    elif 'day' in dh:
        d, h = dh.split(' day, ')
    else:
        d = 0
        h = dh

    secs = int(d)*24*60*60 + int(h)*60*60 + int(m)*60 + int(s)
    return secs

In [4]:
def timetosecond(df):
    df['rec_time'] = df['rec_time'].replace('0', '00:00:00')
    df['rec_time'] = df['rec_time'].apply(to_secs)
    
    return df

In [5]:
train_df = timetosecond(train_df)
valid_df = timetosecond(valid_df)
test_df = timetosecond(test_df)

In [7]:
def time_dff(df):
    
    time_differance_list = []
    
    for i in range(len(df['rec_time'].values)):
        try:
            if df['pid'].values[i] != df['pid'].values[i-1]:
                time_differance_list.append(0)
            else:
                time_differance = df['rec_time'].values[i] - df['rec_time'].values[i-1]
                time_differance_list.append(time_differance)
                
        except:
            time_differance_list.append(0)
        time_dff_df = pd.DataFrame(time_differance_list)
    df.insert(2,'time_dff',time_dff_df)
    
    return df

In [8]:
train_df = time_dff(train_df)
valid_df = time_dff(valid_df)
test_df = time_dff(test_df)

226800 226800
25200 25200
28000 28000


In [20]:
# X, y 분리 및 reshape
y_train = train_df['label']
y_valid = valid_df['label']
y_test = test_df['label']

X_train = train_df.drop(['label'], axis=1)
X_valid = valid_df.drop(['label'], axis=1)
X_test = test_df.drop(['label'], axis=1)



#reshape
maxlen = 70
X_train_3d = X_train.values.reshape((-1, maxlen, X_train.shape[-1]))
X_valid_3d = X_valid.values.reshape((-1, maxlen, X_valid.shape[-1]))
X_test_3d = X_test.values.reshape((-1, maxlen, X_test.shape[-1]))

y_train_3d = y_train[::maxlen]
y_valid_3d = y_valid[::maxlen]
y_test_3d = y_test[::maxlen]